<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>




# Easy object detection with MMDetection

**MMDetection** is a comprehensive toolbox that provides a wide array of object detection algorithms. Developed by the Multimedia Laboratory, CUHK, it's part of the OpenMMLab project.

It's designed to facilitate research and development in object detection, instance segmentation, and other related areas.


![MMDetection illustration](https://miro.medium.com/v2/resize:fit:828/format:webp/1*C3o6c7XBOWpaEfxx0juhKg.jpeg)

## Setup

You need to install Ikomia Python API with pip


In [ ]:
!pip install ikomia

## Run MMPose on your image

---

**-Google Colab ONLY- Restart runtime after the first run of the workflow below** 

Click on the "RESTART RUNTIME" button at the end the previous window.

---

In [ ]:
from ikomia.dataprocess.workflow import Workflow


# Init your workflow
wf = Workflow()

# Add object detection algorithm
detector = wf.add_task(name="infer_mmlab_detection", auto_connect=True)

# Run the workflow on image
wf.run_on(url="https://github.com/open-mmlab/mmdetection/blob/main/demo/demo.jpg?raw=true")

detector.set_parameters({
        "model_name": "rtmdet",
        "model_config": "rtmdet_tiny_8xb32-300e_coco",
        "conf_thres": "0.2",
})

# Get the results
image_output = detector.get_output(0)
detection_output = detector.get_output(1)

In [ ]:
from ikomia.core import IODataType
from ikomia.utils.displayIO import display

from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Display the results
# MMLab detection framework mixes object detection and instance segmentation algorithms
if detection_output.data_type == IODataType.OBJECT_DETECTION:
    display(image_output.get_image_with_graphics(detection_output), title="MMLAB detection")
elif detection_output.data_type == IODataType.INSTANCE_SEGMENTATION:
    display(image_output.get_image_with_mask_and_graphics(detection_output), title="MMLAB detection")

## List of parameters

- **model_name** (str, default="yolox"): model name.
- **model_config** (str, default="yolox_s_8x8_300e_coco"): name of the model configuration file.
- **conf_thres** (float, default=0.5): object detection confidence.
- **use_custom_model** (bool, default=False): flag to enable the custom train model choice.
- **config_file** (str, default=""): path to model config file (only if use_custom_model=True). The file is generated at the end of a custom training. Use algorithm train_mmlab_detection from Ikomia HUB to train custom model.
- **model_weight_file** (str, default=""): path to model weights file (.pt) (only if use_custom_model=True). The file is generated at the end of a custom training.
- **cuda** (bool, default=True): CUDA acceleration if True, run on CPU otherwise.
